# 1. Creació de les classes on es guardes les dades
Es defineixen dues classes:
1. Classe municipi
- Classe partit

La primera classe detalla tots els atributs que conté cada partit polític com vots, el percentatge de vots respecte el total, majoria absoluta, majria relativa i empats. La segona classe detalla tots els atributs que conté cada partit polític com vots, el percentatge de vots respecte el total i regidors.

In [1]:
class municipi:
    # Inicialitzador de la classe
    def __init__(self, nom, url):
        self.__nom = nom
        self.__url = url
        self.partits = []
        self.cens = 0
        self.escrutat = '00,00%'
        self.abstencio = 0
        self.vots_comp = 0
        self.vots_nuls = 0
        self.vots_blanc = 0
    # Getters dels atributs
    def getNom(self):
        return self.__nom
    def getUrl(self):
        return self.__url
    def getPartit(self,index):
        return self.partits[index]
    def getLengthPartits(self):
        return len(self.partits)
    def setPartit(self,partit):
        self.partits.append(partit)
    def setAtributs(self,cens,escrutat,abstencio,vots_comp,vots_nuls,vots_blanc):
        self.cens = cens
        self.escrutat = escrutat
        self.abstencio = abstencio
        self.vots_comp = vots_comp
        self.vots_nuls = vots_nuls
        self.vots_blanc = vots_blanc 
    def getAtributs(self):
        return print('Nom: ',self.nom,' Cens: ',self.cens,' Escrutat: ',self.escrutat,\
                     'Abstencio: ',self.abstencio,'Vots comptabilitats: ',self.vots_comp,\
                     ' Vots nuls: ',self.vots_nuls,' Vots blancs: ',self.vots_blanc)
class partit:
    # Inicialitzador de la classe
    def __init__(self,nom,acronim,vots,percent,regidors):
        self.__nom = nom
        self.__acronim = acronim
        self.__vots = vots
        self.__percent = percent
        self.__regidors = regidors
    # Getters dels atributs
    def getNom(self):
        return self.__nom
    def getAcronim(self):
        return self.__acronim
    def getVots(self):
        return self.__vots
    def getPercent(self):
        return self.__percent
    def getRegidors(self):
        return self.__regidors
    def getTot(self):
        return print('Nom: ',self.__acronim,' Vots: ',self.__vots,' %: ',self.__percent,\
                     ' Regidos: ',self.__regidors)

# 2. Definició i implementació de funcions
A continuació es defineixen i implementen les funcions que s'utilitzen per al procés d'extracció de dades del portal web. 
1. **lecturaWeb**: Realitza una petició al directori web, es comprova quin és el missatge que retorna, i si el codi és '200' vol dir que la petició ha estat entesa i acceptada. En cas contrari, es dona un marge de temps de 5 segons i es torna a provar la petició. Un cop la petició ha estat acceptada, es retorna un elelement del típus BeautifulSoup que conté el contingut del directori web en format html. 

In [2]:
def lecturaWeb(url):
    # INPUT: String que conté la URL on es vol realitzar l'extracció 
    # OUTPUT: Retorna un element SOUP en format html per poder realitzar l'exploració de dades
    
    # Es realitza la petició HTTP
    page = requests.get(url)
    # Es recupera l'estat de la petició i es comprova que sigui correcte
    while page.status_code != 200:
        # En cas que la petició retorni error, s'espera 5 segons i es torna a provar
        print ("Error de connexió del típus {}".format(page.status_code))
        time.sleep(5)
        page = requests.get(url)
    # Es retorna un elemnt típus BeautifulSoup    
    return BeautifulSoup(page.content,"html.parser")

2. **llistaMunicipis**: Funció que cerca el llistat de municipis de les eleccions a Catalunya. 

In [3]:
def llistaMunicipis(llista_municipis,llista_municipis_1,web):
    # INPUT: Directori web on cerca els municipis i llistes municipis buides
    # OUTPUT: Retorna llistes amb tots els municipis i la seva URL associades

    # Càrrega i obtenció de la pàgina web
    soup = lecturaWeb(web)
    
    counter = 1 #variable contador que indica a quina linia es treballa
    for tag in soup.find_all('li', class_ = 'list-group-item'):
        # Els municipis comencen al 47 i acaben al 993. (1-47 provincies i comarques de catalunya i del 994-final C.C.A.A).
        if counter >= 47 and counter <= 993: 
            # S'obté el nom de cada municipi i la seva URL
            nom = tag.a.string # S'accedeix al nom
            url =tag.a['href'] # S'accedeix a la url

            # S'afegueix el municipi a la llista de municipis
            llista_municipis.append(municipi(nom,url))
            llista_municipis_1.append(municipi(nom,url))
        #S'incrementa en 1 el contador
        counter +=1

3. **LecturaMunicipis**: Funció que extreu d'una taula HTML la informació relativa a cada partit polític corresponent a un municipi.

In [4]:
def lecturaMunicipis(tag_taula,municipi):
    # INPUT: tag_taula: tag amb la taula HTML amb els partits
    #        municipi: variable tipus municipi amb la llista de partits buida
    # OUTPUT: municipi modificada ambla llista de partits plena
    
    # llistes nom_partits i acronim partits buides
    noms_partits = []
    acronim_partits = []
    # Llista auxiliar valors buida
    valors = []
    
    # Extracció de nom i l'acrònim de cada partit
    for i in tag_taula.find_all('td',class_ ="linkeleccionsd3 atribpartido"):
        noms_partits.append(i['title'])
        acronim_partits.append(i.findNext(text = True))


    # Extracció dels valors {vots,%,Regidors}, per cada partit->3 valors.
    for i in tag_taula.find_all('td',class_ ="linkeleccionsd3 text-right"):
        valors.append(i.findNext(text = True))

    # Es junta tot
    i = 0
    j = 0

    if len(noms_partits)*3 == len(valors):
        while i<len(valors):
            vots = valors[i]
            i +=1
            percent = valors[i]
            i +=1
            regidors = valors[i]
            i +=1
            partt = partit(noms_partits[j],acronim_partits[j],vots,percent,regidors)
            municipi.setPartit(partt)
            j +=1
    else:
        print("¡¡ERROR!! - Obtenció dades incorrecte")

4. **lecturaAtributs**: Funció que extreu d'una taula HTML els atribut corresponents a cada localitat. La informació que s'extreu és Cens, total escrutat (%), abstencions, vots comptabilitzats, vots nuls i vots en blanc.

In [5]:
def lecturaAtributs(tag,eleccions):
    # INPUT: tag: variable tipus tag que conté la taula amb les dades de cens, etc, del municipi
    #        eleccions: llista de classe municipi   
    # OUTPUT: Variable municipi actualitzada amb els atributs
    
    aux = tag.findNext('td')
    cens = aux.findNext('td')
    aux = cens.findNext('td')
    aux = aux.findNext('td')
    escrutat = aux.findNext('td')
    aux = escrutat.findNext('td')

    vots_comp = aux.findNext('td')
    vots_comp_percent = vots_comp.findNext('td')

    aux = vots_comp_percent.findNext('td')
    abs_comp = aux.findNext('td')
    abs_comp_percent = abs_comp.findNext('td')

    aux = abs_comp_percent.findNext('td')
    vots_nuls = aux.findNext('td')
    vots_nuls_percent = vots_nuls.findNext('td')

    aux = vots_nuls_percent.findNext('td')
    vots_blanc = aux.findNext('td')
    vots_blanc_percent = vots_blanc.findNext('td')
    
    # guarden els atributs extrets a la variable del municipi
    eleccions.setAtributs(cens.string,escrutat.string,abs_comp.string,vots_comp.string,\
                          vots_nuls.string,vots_blanc.string)

5. **extraccioDemarcacio**: Funció que retorna la informació relativa a demarcació > comarca > municipi de cada localitat. 

In [6]:
def extraccioDemarcacio(web):
    # INPUT: url d'on s'extreuen les dades
    # OUTPUT: retorna un dataframe amb la informació de municipi, comarca i provincia per cada localitat
    
    # Definició de l'estructura de dades que conté la iformació de provincia > comarca > municipi
    df_demarcacio = pd.DataFrame(columns=['id_Municipi', 'municipi','comarca', 'provincia'])
    
    # Lectura del directori web per començar l'exploració de dades
    soup =lecturaWeb(web)
    
    a = soup.find(id = 'selectdemarcacio') #Selector Demarcació
    b = soup.find(id = 'selectcomarca') #Selector Comarca
    c = soup.find(id = 'selectmunicipi') #Selector Municipi
    
    # Obtenció del nom de cada municipi i les 'id' de comarca i provincia en format numèric
    counter = 0
    for i in c.find_all('option'):
        if counter != 0:
            df_demarcacio = df_demarcacio.append({'id_Municipi': int(i['value']), 'municipi': i.string,\
                                                  'comarca':i['data-comarca'], 'provincia':i['data-demarcacio']},\
                                                 ignore_index=True)        
        counter += 1
    # Obtenció del nom de cada comarca, es substituira el id pel nom obtingut 
    counter = 0
    for i in b.find_all('option'):
        df_demarcacio['comarca'].replace({i['value']: i.string}, inplace=True)
   
    # Obtenció del nom de cada provincia, es substituira el id pel nom obtingut 
    counter = 0
    for i in a.find_all('option'):
        df_demarcacio['provincia'].replace({i['value']: i.string}, inplace=True)
   
    return df_demarcacio

6. **extraccioDades**: Funció que llegeix la llista de municipis i extreu tota la informació de les eleccions de 2015 i 2019 per municipis.

In [7]:
def extraccioDades(elec_2019,elec_2015):
    # INPUT: elec_20XX: llelec_2019ista de classe municipi amb noms i urls de cadascun ()
    # OUTPUT: elec_20XX: modificada amb els atributs de cada municipi i la llista de partits polítics
    
    counter = 0 # S'inicia contador a 0
    while counter < len(elec_2019):
        
        # Es fa la lectura de URL de cada municipi
        url = elec_2019[counter].getUrl()
        
        # Càrrega i obtenció de la pàgina web
        soup = lecturaWeb(url)
        
        # Obtenció de les taules en format TAG
        taula_2019 = soup.find('table')             # Partits polítics 2019
        dades_2019 = taula_2019.find_next('table')  # Informació extra 2019
        taula_2015 = dades_2019.find_next('table')  # Partits polítics 2015
        dades_2015 = taula_2015.find_next('table')  # Informació extra 2015
              
        # S'extreuen els atributs dels municipis
        lecturaAtributs(dades_2019,elec_2019[counter])
        lecturaAtributs(dades_2015,elec_2015[counter])
        #S'extreuen els partits politics de cada municipi
        lecturaMunicipis(taula_2019,elec_2019[counter])
        lecturaMunicipis(taula_2015,elec_2015[counter])      
        
        counter += 1


# 3. Descarrega de les dades
<p>Un cop definides i implementades les funcions que s'utilitzen,  es procedeix a executar les funcions per a l'extracció de les dades.
<p>En aquest cas, les dades que es volen extreure són els resultats de les eleccions del 2015 i 2019 per municipis a catalunya.

In [8]:
# Càrrega de llibreries necessaries
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup

# Es defineix la web de recerca
web = "https://www.naciodigital.cat/municipals2019"

# Es defineix les llistes de municipis buides. Els municipis són els mateixos per 2015 i 2019.
elec_2019 = []
elec_2015 = []

# Es crida la funció llistaMunicipis()
llistaMunicipis(elec_2019,elec_2015,web)

# S'extreuen les dades
extraccioDades(elec_2019,elec_2015)

# Extracció dades geogràfiques {provincia,comarca,municipi}
# És indiferent la url utilitzada, totes contenen la mateixa informació geogràfica
web1 = ("https://www.naciodigital.cat/municipals2019/municipi/25001/abella/conca") 
# Es crida la funció extraccioDemarcacio
df_demarcacio = extraccioDemarcacio(web1)

Error de connexió del típus 503


# 4. Preparació i extracció de les dades
Es defineixen dos set de dades que contenen tota la informació referents a les eleeccions de 2015 i 2019 per municipis a Catalunya. Els datasets són:
1. ResultatsGlobals
- ResultatsCand

In [9]:
# df_RG correspon al dataframe generat per incloure tota la informació del dataset ResultatsGlobals
# Es defineix el dataframe df_RG
df_RG =  pd.DataFrame(columns=['IdMunicipi', 'NomMunicipi','NomComarca', 'Provincia','URLMunicipi','Cens_2019',\
                                              'Escrutat_2019','Abstencio_2019','Comptabilitzat_2019','Nuls_2019','Blanc_2019',\
                                             'Cens_2015','Escrutat_2015','Abstencio_2015','Comptabilitzat_2015','Nuls_2015',\
                                              'Blanc_2015'])
df_RC = pd.DataFrame(columns=['IdCandidatura','NomCandidatura','AcronimCandidatura','IdMunicipi','NomMunicipi','Vots'\
                              ,'Regidors','Any'])


A continuació, es mostra com es recorre tota l'estructura que conté les dades extretes de la web i s'inclou en els dataframes generats per a ser exportats.

In [10]:
# Sobté ResultatsGlobals
counter = 0
while counter < len(elec_2019):
#while counter < 12:
    # Es troba la coincideincia del dataframe df_demarcacio amb la llista de municipis del any 2019
    aux = df_demarcacio.loc[df_demarcacio['municipi'] == elec_2019[counter].getNom()]
    # Es carreguen el valors individualment
    df_RG = df_RG.append({'IdMunicipi':aux['id_Municipi'].values[0], 'NomMunicipi':aux['municipi'].values[0],\
                          'NomComarca':aux['comarca'].values[0],'Provincia':aux['provincia'].values[0],\
                          'URLMunicipi':elec_2019[counter].getUrl(),'Cens_2019':elec_2019[counter].cens,\
                          'Escrutat_2019':elec_2019[counter].escrutat,'Abstencio_2019':elec_2019[counter].abstencio,\
                          'Comptabilitzat_2019':elec_2019[counter].vots_comp,'Nuls_2019':elec_2019[counter].vots_nuls,\
                          'Blanc_2019':elec_2019[counter].vots_blanc,'Cens_2015':elec_2015[counter].cens,\
                          'Escrutat_2015':elec_2015[counter].escrutat,'Abstencio_2015':elec_2015[counter].abstencio,\
                          'Comptabilitzat_2015':elec_2015[counter].vots_comp,'Nuls_2015':elec_2015[counter].vots_nuls,\
                          'Blanc_2015':elec_2015[counter].vots_blanc},ignore_index=True)  
    
    counter += 1
# Es mostren els primers 5 municipis
df_RG.head(5)


,IdMunicipi,NomMunicipi,NomComarca,Provincia,URLMunicipi,Cens_2019,Escrutat_2019,Abstencio_2019,Comptabilitzat_2019,Nuls_2019,Blanc_2019,Cens_2015,Escrutat_2015,Abstencio_2015,Comptabilitzat_2015,Nuls_2015,Blanc_2015
0,25001,Abella de la Conca,Pallars Jussà,Lleida,https://www.naciodigital.cat/municipals2019/mu...,146,"100,00%",24,122,0,0,140,"100,00%",32,108,3,0
1,8001,Abrera,Baix Llobregat,Barcelona,https://www.naciodigital.cat/municipals2019/mu...,9.100,"100,00%",3.240,5.860,16,32,8.851,"100,00%",3.628,5.223,55,60
2,25003,Agramunt,Urgell,Lleida,https://www.naciodigital.cat/municipals2019/mu...,3.774,"100,00%",1.279,2.495,50,119,3.831,"100,00%",1.453,2.378,37,56
3,8002,Aguilar de Segarra,Bages,Barcelona,https://www.naciodigital.cat/municipals2019/mu...,221,"100,00%",30,191,5,3,203,"100,00%",58,145,10,20
4,17001,Agullana,Alt Empordà,Girona,https://www.naciodigital.cat/municipals2019/mu...,600,"100,00%",130,470,3,12,611,"100,00%",143,468,8,19


In [11]:
# Sobté ResultatsCand
counter = 0
id_Candidatura = 0
while counter < len(elec_2019):
    # Es troba la coincideincia del dataframe df_demarcacio amb la llista de municipis del any 2019
    aux = df_demarcacio.loc[df_demarcacio['municipi'] == elec_2019[counter].getNom()]
    i = 0
    while i < len(elec_2019[counter].partits):
        # Es carreguen el valors individualment
        df_RC = df_RC.append({'IdCandidatura':id_Candidatura,'NomCandidatura':elec_2019[counter].partits[i].getNom(),\
                              'AcronimCandidatura':elec_2019[counter].partits[i].getAcronim(),\
                              'IdMunicipi':aux['id_Municipi'].values[0],\
                              'NomMunicipi':elec_2019[counter].getNom(),'Vots':elec_2019[counter].partits[i].getVots()\
                              ,'Regidors':elec_2019[counter].partits[i].getRegidors(),'Any':int(2019)},ignore_index=True) 
        i += 1
        id_Candidatura += 1  
        
    # Es troba la coincideincia del dataframe df_demarcacio amb la llista de municipis del any 2015
    aux = df_demarcacio.loc[df_demarcacio['municipi'] == elec_2015[counter].getNom()]
    i = 0
    while i < len(elec_2015[counter].partits):
        # Es carreguen el valors individualment
        df_RC = df_RC.append({'IdCandidatura':id_Candidatura,'NomCandidatura':elec_2015[counter].partits[i].getNom(),\
                              'AcronimCandidatura':elec_2015[counter].partits[i].getAcronim(),\
                              'IdMunicipi':aux['id_Municipi'].values[0],\
                              'NomMunicipi':elec_2015[counter].getNom(),'Vots':elec_2015[counter].partits[i].getVots()\
                              ,'Regidors':elec_2015[counter].partits[i].getRegidors(),'Any':int(2015)},ignore_index=True) 
        i += 1
        id_Candidatura += 1             

    
    counter += 1
# Es mostren les primeres 5 candidatures
df_RC.head(5)

,IdCandidatura,NomCandidatura,AcronimCandidatura,IdMunicipi,NomMunicipi,Vots,Regidors,Any
0,0,ESQUERRA REPUBLICANA DE CATALUNYA-ACORD MUNICIPAL,ERC-AM,25001,Abella de la Conca,64,4,2019
1,1,COMPROMÍS PEL MUNICIPI-CANDIDATURA DE PROGRÉS,CM-CP,25001,Abella de la Conca,43,1,2019
2,2,JUNTS PER ABELLA DE LA CONCA,JUNTS,25001,Abella de la Conca,15,0,2019
3,3,JOVES INDEPENDENTS PER ABELLA I BOIXOLS - ACOR...,JIAB - AM,25001,Abella de la Conca,53,4,2015
4,4,INDEPENDENTS PER ABELLA DE LA CONCA - CANDIDATURA,IxAC-CP,25001,Abella de la Conca,34,1,2015


Es realitzen les exportacions dels dataframes a arxius *.csv*.

In [12]:
df_RG.to_csv(r'ResultatsGlobals.csv', index = False, header=True)
df_RC.to_csv(r'ResultatsCand.csv', index = False, header=True)